# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [ ]:
!pip install syft
!pip install numpy

In [ ]:
import torch as th

In [ ]:
x = th.tensor([1,2,3,4,5])
x

In [ ]:
y = x + x

In [ ]:
print(y)

In [ ]:
import syft as sy

In [ ]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [ ]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.send(bob) # send this data to bob

In [ ]:
bob._objects

In [ ]:
x

In [ ]:
x.location

In [ ]:
x.id_at_location

In [ ]:
x.id

In [ ]:
x.owner

In [ ]:
hook.local_worker

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [15]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id="Alice")

In [16]:
# 1 - create some data (a tensor)
y = th.tensor([1,2,3,4,5])
x = y
# 2- send the data to bob and alice
y = y.send(alice,bob)
# x = x.send(bob)

In [ ]:
# notice what does the send fucntion on two workers return; a multi-pointer
#x
y

In [ ]:
# what does .child on the pointer object return?
#y.child
# what does .child.child on the pointer object return? 
y.child.child

In [ ]:
# try the .get() on the pointer
y.get()

In [ ]:
# try the .get(sum_results=True) on your pointer
y.get(sum_results=True)

# Lesson: Introducing Remote Arithmetic

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x # a pointer to a remote tensor located at bob

In [ ]:
y # a pointer to another remote tensor located at bob

In [ ]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

In [ ]:
z = z.get()
z

In [ ]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

In [ ]:
z = z.get() # yup! same results!
z

In [ ]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()
z = z.get()
z

In [ ]:
z.backward() 

In [ ]:
x = x.get()

In [ ]:
x

In [ ]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [1]:
# create some toy data for our model
import syft as sy
import torch
from torch import nn
import torch as th
hook = sy.TorchHook(th)
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True)


W0702 17:23:01.090467 10580 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0702 17:23:01.173463 10580 deprecation_wrapper.py:119] From C:\Users\tklinkenberg\.conda\envs\CS490\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# send the data to bob
bob = sy.VirtualWorker(hook, id="bob")
indata = input_data.send(bob)
outdata = output_data.send(bob)

In [3]:
# create some linear weights and send them to bob4
weights = th.tensor(torch.randn(4,2), requires_grad=True)
weights = weights.send(bob)

C:\Users\tklinkenberg\.conda\envs\CS490\lib\site-packages\syft\frameworks\torch\hook\hook.py:757: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


In [4]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
step1 = torch.mm(indata,weights.view(2,4))
# step2 = torch.mm(step1,outdata)
epochs = 5
for e in range(epochs):
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
    loss = ((step1 - outdata)**2).sum() 
#    3- backpropogate using the backword() function
    loss.backward()
#    4- DO NOT forget to clear your gradients after updating the weights
    # weights.data.sum_(weights.grad * lr)
    # weights = weights.data.sum() * 0


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [5]:
loss.get()

tensor(40.5294, requires_grad=True)

# Lesson: Garbage Collection and Common Errors


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects  # some error from Jupyter is casuing this to appear!

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
bob = bob.clear_objects() # erase force 
bob._objects

In [ ]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects # notice that there is only a single tensor in bob

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [ ]:
z = x + y

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [6]:
from torch import nn, optim

In [7]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [8]:
# A Toy Model
model = nn.Linear(2,1)

In [9]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [10]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(9.2350)
tensor(1.3315)
tensor(0.5550)
tensor(0.3693)
tensor(0.2653)
tensor(0.1934)
tensor(0.1418)
tensor(0.1046)
tensor(0.0775)
tensor(0.0577)
tensor(0.0431)
tensor(0.0323)
tensor(0.0243)
tensor(0.0183)
tensor(0.0139)
tensor(0.0105)
tensor(0.0079)
tensor(0.0060)
tensor(0.0046)
tensor(0.0035)


In [11]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [17]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [18]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [19]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [20]:
train()

tensor(2.8233, requires_grad=True)
tensor(0.4047, requires_grad=True)
tensor(0.3651, requires_grad=True)
tensor(0.2801, requires_grad=True)
tensor(0.1973, requires_grad=True)
tensor(0.1669, requires_grad=True)
tensor(0.1206, requires_grad=True)
tensor(0.0996, requires_grad=True)
tensor(0.0750, requires_grad=True)
tensor(0.0602, requires_grad=True)
tensor(0.0473, requires_grad=True)
tensor(0.0370, requires_grad=True)
tensor(0.0302, requires_grad=True)
tensor(0.0233, requires_grad=True)
tensor(0.0196, requires_grad=True)
tensor(0.0149, requires_grad=True)
tensor(0.0128, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0014, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [21]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:Alice #objects:0>

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
alice._objects

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
alice._objects